In [1]:
! pip install gradio

import pandas as pd
import re
import json
import os
import datetime
import joblib
import gradio as gr
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# Load and preprocess dataset
df = pd.read_csv("/content/medimind_india_raw_data.csv")
df.dropna(subset=["symptoms", "disease", "treatment_plan"], inplace=True)
df.reset_index(drop=True, inplace=True)

In [3]:
def clean_text(text):
    return re.sub(r"[^\w\s]", "", text.lower()).strip()

# Clean symptoms and encode disease labels
df["symptoms"] = df["symptoms"].apply(clean_text)
df["disease"] = df["disease"].astype(str)

label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["disease"])
num_labels = len(label_encoder.classes_)

In [4]:
# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
# Paths for model and encoder
model_path = "disease_model"
encoder_path = "label_encoder.joblib"

In [6]:
# Load or train model
if os.path.exists(model_path) and os.path.exists(encoder_path):
    model = TFDistilBertForSequenceClassification.from_pretrained(model_path)
    label_encoder = joblib.load(encoder_path)
else:
    model = TFDistilBertForSequenceClassification.from_pretrained(
        "distilbert-base-uncased", num_labels=num_labels
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )
    encodings = tokenizer(df["symptoms"].tolist(), truncation=True, padding=True, max_length=128, return_tensors="tf")
    dataset = tf.data.Dataset.from_tensor_slices((dict(encodings), df["label"].values)).shuffle(len(df)).batch(16)
    train_size = int(0.8 * len(dataset))
    model.fit(dataset.take(train_size), validation_data=dataset.skip(train_size), epochs=3)
    model.save_pretrained(model_path)
    joblib.dump(label_encoder, encoder_path)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3
500/500 [==============================] - 74s 58ms/step - loss: 0.1999 - accuracy: 0.9668 - val_loss: 0.0264 - val_accuracy: 0.9930
Epoch 2/3
500/500 [==============================] - 31s 63ms/step - loss: 0.0213 - accuracy: 0.9931 - val_loss: 0.0195 - val_accuracy: 0.9950
Epoch 3/3
500/500 [==============================] - 24s 49ms/step - loss: 0.0163 - accuracy: 0.9935 - val_loss: 0.0140 - val_accuracy: 0.9965


In [18]:
def clinical_decision(disease):
    if "infarction" in disease.lower() or "cardiac" in disease.lower():
        return {
            "symptom_analysis": {
                "differential_diagnoses": [
                    {"name": "Acute myocardial infarction (STEMI)", "confidence": "92%"},
                    {"name": "Unstable angina", "confidence": "5%"},
                    {"name": "Aortic dissection", "confidence": "2%"},
                    {"name": "Pulmonary embolism", "confidence": "1%"}
                ],
                "alert": "Immediate cardiology consult recommended."
            },
            "decision_support": {
                "immediate_actions": [
                    "Aspirin 325 mg (chewable) stat.",
                    "Heparin bolus and infusion.",
                    "Prepare for PCI within 90 minutes."
                ],
                "follow_ups": [
                    "Repeat ECG in 30 minutes.",
                    "Monitor troponin every 6 hours."
                ],
                "documentation": "STEMI diagnosed; aspirin and heparin initiated; PCI planned."
            }
        }
    elif "diabetes" in disease.lower():
        return {
            "symptom_analysis": {
                "differential_diagnoses": [
                    {"name": "Type 2 diabetes mellitus", "confidence": "90%"},
                    {"name": "Type 1 diabetes", "confidence": "5%"},
                    {"name": "Hyperthyroidism", "confidence": "3%"},
                    {"name": "Malignancy", "confidence": "2%"}
                ],
                "alert": "Monitor glucose and refer for diet and lifestyle counseling."
            },
            "decision_support": {
                "immediate_actions": [
                    "Metformin 500 mg once daily.",
                    "Refer to dietician for low-carb diet."
                ],
                "follow_ups": [
                    "Repeat HbA1c in 3 months.",
                    "Order lipid panel and kidney tests."
                ],
                "documentation": "Start metformin and follow a healthy diet."
            }
        }
    else:
        return {
            "symptom_analysis": {
                "differential_diagnoses": [{"name": disease, "confidence": "80%"}],
                "alert": "Further evaluation recommended."
            },
            "decision_support": {
                "immediate_actions": ["Supportive care"],
                "follow_ups": ["Follow-up in 1 week"],
                "documentation": f"Diagnosis: {disease}"
            }
        }


In [19]:
def analyze_patient(name, age, gender, symptoms, history, tests, query):
    input_text = f"{symptoms} {history} {tests} {query}".lower()
    inputs = tokenizer(input_text, return_tensors="tf", truncation=True, padding=True, max_length=128)
    logits = model(inputs).logits
    prediction = tf.argmax(logits, axis=1).numpy()[0]
    predicted_disease = label_encoder.inverse_transform([prediction])[0]
    decision = clinical_decision(predicted_disease)

    patient_record = {
        "timestamp": str(datetime.datetime.now()),
        "name": name,
        "age": age,
        "gender": gender,
        "input": {
            "symptoms": symptoms,
            "history": history,
            "tests": tests,
            "query": query
        },
        "predicted_disease": predicted_disease,
        "outputs": decision
    }

    # Save to files
    with open("patient_records.jsonl", "a") as f_json:
        f_json.write(json.dumps(patient_record) + "\n")

    df_path = "patient_records.csv"
    csv_row = {
        "timestamp": patient_record["timestamp"],
        "name": name,
        "age": age,
        "gender": gender,
        "symptoms": symptoms,
        "history": history,
        "tests": tests,
        "query": query,
        "predicted_disease": predicted_disease
    }
    df_new = pd.DataFrame([csv_row])
    if not os.path.exists(df_path):
        df_new.to_csv(df_path, index=False)
    else:
        df_existing = pd.read_csv(df_path)
        df_all = pd.concat([df_existing, df_new], ignore_index=True)
        df_all.to_csv(df_path, index=False)

    # Format output like MediMind clinical examples
    readable_output = f"""## Clinical Summary for {name} ({age}, {gender})\n
**Input**:
- Symptoms: {symptoms}
- Medical History: {history}
- Test Results: {tests}
- Physician Query: {query}

**Output (Symptom Analysis)**:
""" + "\n".join([f"- {d['name']}: {d['confidence']}" for d in decision['symptom_analysis']['differential_diagnoses']]) + \
f"""\n- Alert: {decision['symptom_analysis']['alert']}

**Output (Clinical Decision Support)**:
- Immediate Actions:
""" + "\n".join([f"  - {act}" for act in decision['decision_support']['immediate_actions']]) + \
f"""\n- Follow-Ups:\n""" + "\n".join([f"  - {f}" for f in decision['decision_support']['follow_ups']]) + \
f"""\n- Documentation: {decision['decision_support']['documentation']}"""

    return predicted_disease, readable_output


In [21]:
def run_gradio():
    interface = gr.Interface(
        fn=analyze_patient,
        inputs=[
            gr.Textbox(label="Patient Name"),
            gr.Textbox(label="Age"),
            gr.Radio(["Male", "Female", "Other"], label="Gender"),
            gr.Textbox(label="Symptoms", lines=2, placeholder="E.g., chest pain, sweating..."),
            gr.Textbox(label="Medical History", lines=2),
            gr.Textbox(label="Test Results", lines=2),
            gr.Textbox(label="Physician Query", lines=2, placeholder="What’s the likely diagnosis, and what should we do?")
        ],
        outputs=[
            gr.Textbox(label="Predicted Disease"),
            gr.Textbox(label="Detailed Report", lines=20)
        ],
        title=" MediMind - AI Medical Assistant",
        description="Enter patient details to get disease prediction and clinical decision support.",
        allow_flagging="never"
    )
    interface.launch(share=True)
run_gradio()


/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://04cc4cc927d9bd7219.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
